In [3]:
from datasets import load_dataset, Dataset
import re

# Load the dataset
dataset = load_dataset('sinarashidi/temp')

# Shuffle the dataset and slice it
# dataset = dataset['train'].shuffle(seed=42).select(range(len(dataset)))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': '\\\n'.join(reformatted_segments)}

# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


Map: 100%|██████████| 93921/93921 [00:01<00:00, 59701.55 examples/s]


In [4]:
# transformed_dataset.save_to_disk('./')
transformed_dataset.push_to_hub("sinarashidi/sentiment-claim-stance")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  3.60it/s]


In [ ]:
import json

output = []
with open('./data/sentiment_claim_stance.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        output.append(data)

len(output)

In [ ]:
out1 = []
out2 = []
out3 = []

for i in range(5571):
    data = output[i]
    d = data['text']
    modified = d.replace("Perform stance detection on the following sentence and its reply sentence:", "Stance Detection:")
    data['text'] = modified
    out1.append(data)

for i in range(5571, 9440):
    data = output[i]
    d = data['text']
    modified = d.replace("Perform claim detection on the following sentence:", "Claim Detection:")
    data['text'] = modified
    out2.append(data)

for i in range(9440, 93921):
    data = output[i]
    d = data['text']
    modified = d.replace("Detect the sentiment of this comment.", "Sentiment Detection:")
    data['text'] = modified
    out3.append(data)

In [ ]:
type(out1[1])

In [ ]:
with open('./output.jsonl', 'w') as f:
    for d in out1:
        json.dump(d, f, ensure_ascii=False)
        f.write('\n')
    for d in out2:
        json.dump(d, f, ensure_ascii=False)
        f.write('\n')
    for d in out3:
        json.dump(d, f, ensure_ascii=False)
        f.write('\n')